In [61]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# %cd ..
!ls
from core import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Data        README.md   core        models      submit.sh
Notebooks   __init__.py depricated  notes_ideas train.py


Similar to mnist GAN, 

mnist was a dataset of shape (n, 28, 28)

ddsp is a dataset of shape (n, 64000)

#### Discriminator will be 

input shape (64000)

encode into latent space (6 x 1000)

- f0_scaled (1 x 1000)
- ld_scaled (1 x 1000)
- z (4 x 1000)

output shape (1)

In [161]:
# get a dataset
output_tfrecord_path = './Data/piano/piano30s.tfrecord'
dataset = DDSP_DATASET(output_tfrecord_path)

# get a sample
i = 0
real_sample = dataset.getSample(sampleNum=i)
# expand the audio dimension to work with z encoder 
real_sample['audio'] = tf.expand_dims(real_sample['audio'],axis=[0]) 
describeSample(real_sample)

audio: (1, 64000) Range: -0.24915089 0.28982782
f0_confidence: (1000,) Range: 0.019729322 0.9687585
f0_hz: (1000,) Range: 41.480316 783.93646
loudness_db: (1000,) Range: -84.711914 -58.525723


In [162]:
# first get into f0_scaled and ld_scaled 
preprocessor = ddsp.training.preprocessing.DefaultPreprocessor(time_steps=1000)
preprocessed = preprocessor(real_sample, training=True)
describeSample(preprocessed)

audio: (1, 64000) Range: -0.24915089 0.28982782
f0_confidence: (1000,) Range: 0.019729322 0.9687585
f0_hz: (1, 1000, 1) Range: 41.480316 783.93646
loudness_db: (1, 1000, 1) Range: -84.711914 -58.525723
f0_scaled: (1, 1000, 1) Range: 0.22138539 0.6220379
ld_scaled: (1, 1000, 1) Range: 0.29406738 0.51228565


In [159]:
# get z :::TODO we aren't GENERATING z to get fake sample......
encoder = ddsp.training.encoders.MfccTimeDistributedRnnEncoder(z_dims=6,z_time_steps=1000)
encoded = encoder(preprocessed)
describeSample(encoded)

audio: (1, 64000) Range: -0.24915089 0.28982782
f0_confidence: (1000,) Range: 0.019729322 0.9687585
f0_hz: (1, 1000, 1) Range: 41.480316 783.93646
loudness_db: (1, 1000, 1) Range: -84.711914 -58.525723
f0_scaled: (1, 1000, 1) Range: 0.22138539 0.6220379
ld_scaled: (1, 1000, 1) Range: 0.29406738 0.51228565
z: (1, 1000, 6) Range: -0.7080601 0.9048021


In [160]:
# input_dim = (None, 1000, 6, 1)
encoded_concat = tf.concat([encoded['f0_scaled'],encoded['ld_scaled'],encoded['z']],axis=2)
encoded_concat = tf.expand_dims(encoded_concat,axis=3)
encoded_concat.shape

TensorShape([1, 1000, 8, 1])

In [148]:
# now encode even further down to a binary classification real or fake
discriminator = Sequential()
# downsample to 500x3
discriminator.add(Conv2D(16, (3,3), strides=(2, 2), padding='same', input_shape=(1000,6,1)))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(alpha=0.2))
# downsample to 250 x 2
discriminator.add(Conv2D(16, (3,3), strides=(2, 2), padding='same', input_shape=(1000,6,1)))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(alpha=0.2))
# downsample to 125 x 1
discriminator.add(Conv2D(16, (3,3), strides=(2, 2), padding='same', input_shape=(1000,6,1)))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(alpha=0.2))
# classify
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
discriminator.summary()

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 500, 3, 16)        160       
_________________________________________________________________
batch_normalization_107 (Bat (None, 500, 3, 16)        64        
_________________________________________________________________
leaky_re_lu_107 (LeakyReLU)  (None, 500, 3, 16)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 250, 2, 16)        2320      
_________________________________________________________________
batch_normalization_108 (Bat (None, 250, 2, 16)        64        
_________________________________________________________________
leaky_re_lu_108 (LeakyReLU)  (None, 250, 2, 16)        0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 125, 1, 16)      

In [152]:
discriminator(encoded_concat)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.49385604]], dtype=float32)>

#### Generator

input shape (100)

upsample into (f0_scaled, ld_scaled) (2x1000) 

run through decoder & processor group

output shape (64000)

In [25]:
from tensorflow.keras.utils import plot_model

In [155]:
generator = Generator(latent_dim=100)
fake_sample = generator.generate()
describeSample(fake_sample)
play(fake_sample['audio'])

# params:  1276365

f0_scaled: (1, 1000, 1) Range: 0.4763164 0.5022575
ld_scaled: (1, 1000, 1) Range: 0.46359998 0.49775836
z: (1, 1000, 6) Range: 0.45113227 0.5074742
f0_hz: (1, 1000, 1) Range: 269.17023 325.5913
loudness_db: (1, 1000, 1) Range: -64.368004 -60.268993
amps: (1, 1000, 1) Range: -0.41246575 -0.17274076
harmonic_distribution: (1, 1000, 45) Range: -2.584573 2.02551
noise_magnitudes: (1, 1000, 45) Range: -2.0264444 1.9129786
audio: (1, 64000) Range: -0.26198536 0.26551598
